In [ ]:
import os
import math
import random
import copy
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from collections import Counter, defaultdict
from itertools import chain, combinations
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (
    accuracy_score, 
    confusion_matrix, 
    ConfusionMatrixDisplay, 
    classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
from Nash import find_nash_equilibria_v2
from Functions import compute_and_visualize_shapley_values


In [3]:
# disable ConvergenceWarnings
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [4]:
# PRNG seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

In [5]:
def aggregate_lr_models(models: list):
    aggregated_model = LogisticRegression()
    aggregated_model.coef_ = np.mean([model.coef_ for model in models], axis=0)
    noise = np.random.normal(0, 0.05, aggregated_model.coef_.shape)
    aggregated_model.coef_ += noise
    aggregated_model.intercept_ = np.mean([model.intercept_ for model in models], axis=0)
    aggregated_model.classes_ = models[0].classes_
    return aggregated_model

In [ ]:
file_path = '.../Thesis_Project_Spambase/data/spambase.data'  # Adjust the path as needed
df = pd.read_csv(file_path, header=None)

In [7]:
random_seed= 42
X = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
def split_data_equal(X: np.ndarray, y: np.ndarray, n_clients: int, shuffle: bool=False, random_seed: int = None):
    if shuffle:
        if random_seed is not None:
            np.random.seed(random_seed)
        idx = np.random.permutation(X.shape[0])
        X = X[idx]
        y = y[idx]

    n_entries_per_client = X.shape[0] // n_clients
    partitions = []
    for i in range(n_clients):
        start = i * n_entries_per_client
        end = (i + 1) * n_entries_per_client
        X_i = X[start:end]
        y_i = y[start:end]
        partitions.append((X_i, y_i))
    # Ensure the last partition gets all remaining data.
    partitions[-1] = (X[start:], y[start:])

    return partitions


In [9]:
def corrupt_data(X: np.ndarray, y: np.ndarray, corruption_prob: float = 0.8, nan_prob: float = 0.5, noise_std: Optional[float] = None):
    if noise_std is None:
        noise_std = 0.1
        
    X = X.copy()

    mask = np.random.rand(X.shape[0]) < corruption_prob
    mask_nan = mask & (np.random.rand(X.shape[0]) < nan_prob)
    mask_noise = mask & (~mask_nan)

    X += mask_noise[:, None] * np.random.randn(X.shape[0], X.shape[1]) * 0.3
    X[np.argwhere(mask_nan)] = np.nan
    
    return X, y



# Function to corrupt data for specific clients, keeping the same corruption indices across trials
def corrupt_clients(f, partitions, corrupt_client_indices):

    for idx in corrupt_client_indices:
        X_i, y_i = partitions[idx]
        X_i_corrupted, y_i_corrupted = f(X_i.copy(), y_i.copy())
        partitions[idx] = (X_i_corrupted, y_i_corrupted)
    
    return partitions, corrupt_client_indices

In [10]:
# Split data among clients
partitions = split_data_equal(X_train, y_train, n_clients=10, shuffle=True)

In [11]:
def train_client_models(partitions, random_seed, X_test, y_test, max_iter):
    client_models = []
    client_global_accuracies = []
    client_local_accuracies = []
    for X_i, y_i in partitions:
        # Remove rows with NaN values
        nan_mask = ~np.isnan(X_i).any(axis=1)
        X_i_clean = X_i[nan_mask]
        y_i_clean = y_i[nan_mask]
        if len(y_i_clean) == 0:
            # No data to train on after cleaning
            client_models.append(None)
            client_global_accuracies.append(None)
            client_local_accuracies.append(None)
            continue
        model = LogisticRegression(random_state=random_seed, max_iter=max_iter)
        try:
            model.fit(X_i_clean, y_i_clean)
            client_models.append(model)

            acc_local = model.score(X_i_clean, y_i_clean)
            client_local_accuracies.append(acc_local)

            # Compute local accuracy on global test set
            local_acc = model.score(X_test, y_test)
            client_global_accuracies.append(local_acc)

        except Exception as e:
            # Model training failed
            client_models.append(None)
            client_global_accuracies.append(None)
            client_local_accuracies.append(None)
    return client_models, client_local_accuracies, client_global_accuracies

def run_federated_learning(
    partitions_corrupted,  # Precomputed corrupted partitions
    corrupt_client_indices,
    n_clients=10,
    random_seed=None,
    shuffle=True,
    max_iter=10,
):


    # Train models for each client and compute accuracies
    client_models, client_local_accuracies, client_global_accuracies = train_client_models(
        partitions_corrupted, random_seed, X_test, y_test, max_iter
    )

    # Generate all possible combinations of clients represented as binary strings
    results = []
    n_combinations = 2 ** n_clients
    for i in range(1, n_combinations):
        bin_str = format(i, f'0{n_clients}b')
        # Determine which clients are included in this combination
        client_indices = [j for j in range(n_clients) if bin_str[n_clients - 1 - j] == '1']
        included_models = [client_models[j] for j in client_indices if client_models[j] is not None]

        if not included_models:
            # Skip combinations where no valid models are available
            continue

        # Aggregate models
        federated_model = aggregate_lr_models(included_models)
        agg_acc = federated_model.score(X_test, y_test)

        # Prepare the row data
        row = {'Combination': bin_str, 'Clients': [client + 1 for client in client_indices], 'Global Accuracy': agg_acc}

        # Add global accuracies for each client
        for j in range(n_clients):
            acc = client_global_accuracies[j] if client_global_accuracies[j] is not None else np.nan
            column_name = f"Client {j + 1} Accuracy"
            if j in corrupt_client_indices:
                column_name += " (corrupted client)"
            row[column_name] = acc

        results.append(row)

    df_results = pd.DataFrame(results)

    print(f"Corrupted client indices: {np.array(corrupt_client_indices) + 1}")

    return df_results, corrupt_client_indices, client_local_accuracies, client_global_accuracies


In [28]:
def run_experiment(n_trials, n_clients, max_iters, partitions_corrupted, corrupt_client_indices, df, save_dir):
    os.makedirs(save_dir, exist_ok=True)  # Ensure save directory exists

    all_nash_details = []

    for max_iter in max_iters:
        print(f"\nRunning experiment for max_iter = {max_iter}")
        nash_counts = Counter()
        details_for_this_max_iter = []

        # Fix corruption indices across all trials
        if corrupt_client_indices is None or len(corrupt_client_indices) == 0:
            corrupt_client_indices = np.random.choice(n_clients, size=partitions_corrupted, replace=False)


        for trial in range(n_trials):
            print(f"  Running trial {trial + 1}/{n_trials} for max_iter = {max_iter}")

            base_random_seed = 42
            trial_seed = base_random_seed + trial + max_iter
            random.seed(trial_seed)
            np.random.seed(trial_seed)

            # Split data
            X = df.iloc[:, :-1].to_numpy()
            y = df.iloc[:, -1].to_numpy()
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=trial_seed)

            # Normalize data
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            # Split data into client partitions
            partitions = split_data_equal(X_train, y_train, n_clients=n_clients, shuffle=True, random_seed=trial_seed)
            
            # Apply corruption using fixed corrupt client indices
            corrupted_partitions, _ = corrupt_clients(corrupt_data, partitions, corrupt_client_indices)

            # Run federated learning with fixed corruption
            df_results, _, client_local_accuracies, client_global_accuracies = run_federated_learning(
                partitions_corrupted=corrupted_partitions,
                corrupt_client_indices=corrupt_client_indices,
                n_clients=n_clients,
                random_seed=random_seed + trial,
                max_iter=max_iter,
            )

            # Find Nash equilibria
            df_nash = find_nash_equilibria_v2(df_results)

            # Count Nash equilibrium occurrences
            for coalition in df_nash['Combination']:
                nash_counts[coalition] += 1

            # Compute Shapley values
            client_local_accuracies_dict = {i: client_local_accuracies[i] for i in range(len(client_local_accuracies))}
            df_shapley = compute_and_visualize_shapley_values(df_results, client_local_accuracies_dict, n_clients=n_clients, plot=False, print_df=False)
            df_shapley['Trial'] = trial + 1

            # Save details for each Nash equilibrium
            for idx, nash_row in df_nash.iterrows():
                combination_bin = nash_row['Combination']
                client_indices = [j for j in range(n_clients) if combination_bin[n_clients - 1 - j] == '1']
                clients_list = [j + 1 for j in client_indices]
                
                detail_record = {
                    'Trial': trial + 1,
                    'Combination': combination_bin,
                    'Clients': ",".join(str(c) for c in clients_list),
                    'Global Accuracy': nash_row['Global Accuracy'],
                    'Max Iter': max_iter
                }

                for client in range(1, n_clients + 1):
                    if client in clients_list:
                        detail_record[f'Client {client} Accuracy'] = client_global_accuracies[client - 1]
                        val = df_shapley.loc[df_shapley['Client'] == f'Client {client}', 'Normalized Shapley Value']
                        detail_record[f'Client {client} Shapley Value'] = val.values[0] if not val.empty else np.nan
                    else:
                        detail_record[f'Client {client} Accuracy'] = np.nan
                        detail_record[f'Client {client} Shapley Value'] = np.nan

                details_for_this_max_iter.append(detail_record)

        # Save Nash Equilibrium counts
        df_nash_counts = pd.DataFrame(nash_counts.items(), columns=['Nash Equilibrium', 'Occurrences'])
        df_nash_counts['Max Iter'] = max_iter
        filename_counts = os.path.join(save_dir, f"Nash_Equilibrium_Counts_SpambaseLR_maxiter_{max_iter}.csv")
        df_nash_counts.to_csv(filename_counts, index=False)
        print(f"Nash equilibrium counts saved to {filename_counts}")

        # Save detailed results
        df_details = pd.DataFrame(details_for_this_max_iter)
        all_nash_details.append(df_details)

    # Save final Nash Equilibrium details
    final_details_df = pd.concat(all_nash_details, ignore_index=True)
    final_details_path = os.path.join(save_dir, "Nash_Equilibrium_Details_SpambaseLR.csv")
    final_details_df.to_csv(final_details_path, index=False)
    print(f"Nash equilibrium details saved to {final_details_path}")

    return final_details_df


In [ ]:
save_dir='.../results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedLR/3_Corrupted_Clients'
results = run_experiment(n_trials=10,  n_clients=10, max_iters=[10,100], partitions_corrupted= 3, corrupt_client_indices= [0,1,2], df=df, save_dir=save_dir)


Running experiment for max_iter = 10
  Running trial 1/10 for max_iter = 10
Corrupted client indices: [1 2 3]
  Running trial 2/10 for max_iter = 10
Corrupted client indices: [1 2 3]
  Running trial 3/10 for max_iter = 10
Corrupted client indices: [1 2 3]
  Running trial 4/10 for max_iter = 10
Corrupted client indices: [1 2 3]
  Running trial 5/10 for max_iter = 10
Corrupted client indices: [1 2 3]
  Running trial 6/10 for max_iter = 10
Corrupted client indices: [1 2 3]
  Running trial 7/10 for max_iter = 10
Corrupted client indices: [1 2 3]
  Running trial 8/10 for max_iter = 10
Corrupted client indices: [1 2 3]
  Running trial 9/10 for max_iter = 10
Corrupted client indices: [1 2 3]
  Running trial 10/10 for max_iter = 10
Corrupted client indices: [1 2 3]
Nash equilibrium counts saved to /Users/abbaszal/Documents/Thesis_Project_Spambase/results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedLR/3_Corrupted_Clients/Nash_Equilibrium_Counts_SpambaseLR_maxiter_10.csv

Running exper

In [ ]:
save_dir='.../results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedLR/4_Corrupted_Clients'
results = run_experiment(n_trials=10,  n_clients=10, max_iters=[10,100], partitions_corrupted= 4, corrupt_client_indices= [0,1,2,3], df=df, save_dir=save_dir)


Running experiment for max_iter = 10
  Running trial 1/10 for max_iter = 10
Corrupted client indices: [1 2 3 4]
  Running trial 2/10 for max_iter = 10
Corrupted client indices: [1 2 3 4]
  Running trial 3/10 for max_iter = 10
Corrupted client indices: [1 2 3 4]
  Running trial 4/10 for max_iter = 10
Corrupted client indices: [1 2 3 4]
  Running trial 5/10 for max_iter = 10
Corrupted client indices: [1 2 3 4]
  Running trial 6/10 for max_iter = 10
Corrupted client indices: [1 2 3 4]
  Running trial 7/10 for max_iter = 10
Corrupted client indices: [1 2 3 4]
  Running trial 8/10 for max_iter = 10
Corrupted client indices: [1 2 3 4]
  Running trial 9/10 for max_iter = 10
Corrupted client indices: [1 2 3 4]
  Running trial 10/10 for max_iter = 10
Corrupted client indices: [1 2 3 4]
Nash equilibrium counts saved to /Users/abbaszal/Documents/Thesis_Project_Spambase/results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedLR/4_Corrupted_Clients/Nash_Equilibrium_Counts_SpambaseLR_maxiter_1

In [ ]:
save_dir='.../results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedLR/6_Corrupted_Clients'
results = run_experiment(n_trials=10,  n_clients=10, max_iters=[10,100], partitions_corrupted= 6, corrupt_client_indices= [0,1,2,3,4,5], df=df, save_dir=save_dir)


Running experiment for max_iter = 10
  Running trial 1/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6]
  Running trial 2/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6]
  Running trial 3/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6]
  Running trial 4/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6]
  Running trial 5/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6]
  Running trial 6/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6]
  Running trial 7/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6]
  Running trial 8/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6]
  Running trial 9/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6]
  Running trial 10/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6]
Nash equilibrium counts saved to /Users/abbaszal/Documents/Thesis_Project_Spambase/results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedLR/6_Corrupted_Clients/Nash

In [ ]:
save_dir='.../results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedLR/8_Corrupted_Clients'
results = run_experiment(n_trials=10,  n_clients=10, max_iters=[10,100], partitions_corrupted= 8, corrupt_client_indices= [0,1,2,3,4,5,6,7], df=df, save_dir=save_dir)


Running experiment for max_iter = 10
  Running trial 1/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]
  Running trial 2/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]
  Running trial 3/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]
  Running trial 4/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]
  Running trial 5/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]
  Running trial 6/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]
  Running trial 7/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]
  Running trial 8/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]
  Running trial 9/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]
  Running trial 10/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]
Nash equilibrium counts saved to /Users/abbaszal/Documents/Thesis_Project_Spambase/results/fedLR_FedFor__with_Corrupting_Spambase_N

In [ ]:
save_dir='.../results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedLR/9_Corrupted_Clients'
results = run_experiment(n_trials=10,  n_clients=10, max_iters=[10,100], partitions_corrupted= 9, corrupt_client_indices= [0,1,2,3,4,5,6,7,8], df=df, save_dir=save_dir)


Running experiment for max_iter = 10
  Running trial 1/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]
  Running trial 2/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]
  Running trial 3/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]
  Running trial 4/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]
  Running trial 5/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]
  Running trial 6/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]
  Running trial 7/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]
  Running trial 8/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]
  Running trial 9/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]
  Running trial 10/10 for max_iter = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]
Nash equilibrium counts saved to /Users/abbaszal/Documents/Thesis_Project_Spambase/results/fedLR_FedFor__with_C

### FedFor

In [20]:
# Define the DecisionTree class
class DecisionTree:
    def __init__(self, max_depth=None, random_state=None):
        self.max_depth = max_depth
        self.tree_ = None
        self.random_state = random_state

    def fit(self, X, y):
        self.tree_ = DecisionTreeClassifier(max_depth=self.max_depth, random_state=self.random_state)
        self.tree_.fit(X, y)

    def predict(self, X):
        return self.tree_.predict(X)

# Define the FederatedForest class
class FederatedForest:
    def __init__(self):
        self.models = []

    def add_model(self, model):
        self.models.append(model)

    def predict(self, X):
        predictions = [model.predict(X) for model in self.models]
        predictions = np.array(predictions).T
        y = [np.bincount(row).argmax() for row in predictions]
        return np.array(y)
    

In [21]:
def run_experiment_2(n_trials, n_clients, max_depths, partitions_corrupted, corrupt_client_indices, df, save_dir="results", base_random_seed=42):
    os.makedirs(save_dir, exist_ok=True)

    all_results = []             
    all_nash_equilibrium_counts = [] 
    all_nash_details = []            

    for max_depth in max_depths:
        print(f"\nRunning experiment with max_depth = {max_depth}")

        results = []             
        nash_equilibrium_counts = Counter()
        nash_details_all = []    

        # Fix corruption indices across all trials
        if corrupt_client_indices is None or len(corrupt_client_indices) == 0:
            corrupt_client_indices = np.random.choice(n_clients, size=partitions_corrupted, replace=False)

        for trial in range(n_trials):
            print(f"  Trial {trial + 1}/{n_trials} for max_depth = {max_depth}")
            trial_seed = base_random_seed + trial + max_depth
            random.seed(trial_seed)
            np.random.seed(trial_seed)

            print(f"Corrupted client indices: {np.array(corrupt_client_indices) + 1}")

            # Split data
            target_column = df.columns[-1]
            X = df.drop(target_column, axis=1).to_numpy()
            y = df[target_column].to_numpy()
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=trial_seed)

            # Normalize data
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            # Partition data among clients
            partitions = split_data_equal(X_train, y_train, n_clients=n_clients, shuffle=True, random_seed=trial_seed)

            # Apply corruption using fixed corrupt client indices
            corrupted_partitions, _ = corrupt_clients(corrupt_data, partitions, corrupt_client_indices)

            # Train models for each client
            client_models = []
            client_global_accuracies = {}  
            for i, (X_i, y_i) in enumerate(corrupted_partitions):
                model = DecisionTreeClassifier(max_depth=max_depth, random_state=np.random.randint(0, 100000))
                model.fit(X_i, y_i)
                client_models.append(model)

                y_pred = model.predict(X_test)
                acc_global = accuracy_score(y_test, y_pred)
                client_global_accuracies[i] = acc_global

            trial_results = []
            n_combinations = 2 ** n_clients
            for i in tqdm(range(1, n_combinations), desc=f"    Evaluating coalitions for trial {trial + 1}"):
                bin_str = format(i, f'0{n_clients}b')
                coalition_indices = [j for j in range(n_clients) if bin_str[n_clients - 1 - j] == '1']
                included_models = [client_models[j] for j in coalition_indices]
                if not included_models:
                    continue

                # Federated model aggregation
                forest = FederatedForest()
                for model in included_models:
                    forest.add_model(model)
                y_pred_global = forest.predict(X_test)
                coalition_acc_global = accuracy_score(y_test, y_pred_global)

                row = {
                    'Trial': trial + 1,
                    'Combination': bin_str,
                    'Clients': ",".join(str(j + 1) for j in coalition_indices),
                    'Global Accuracy': coalition_acc_global
                }

                for j in range(n_clients):
                    row[f'Client {j + 1} Accuracy'] = client_global_accuracies.get(j, np.nan)
                trial_results.append(row)

            df_trial_results = pd.DataFrame(trial_results)
            df_trial_results.set_index('Combination', inplace=True)
            # Fix 'Clients' column before computing Shapley values
            df_trial_results['Clients'] = df_trial_results['Clients'].apply(
                lambda x: list(map(int, x.split(','))) if isinstance(x, str) else x
            )

            # Find Nash equilibria
            nash_df = find_nash_equilibria_v2(df_trial_results.reset_index())
            for combination in nash_df['Combination']:
                nash_equilibrium_counts[combination] += 1

            # Compute Shapley values
            shapley_df = compute_and_visualize_shapley_values(
                df_trial_results.reset_index(), client_global_accuracies, n_clients=n_clients, plot=False, print_df=False
            )
            shapley_df['Trial'] = trial + 1

            # Save details for each Nash equilibrium
            for idx, nash_row in nash_df.iterrows():
                combination_bin = nash_row['Combination']
                coalition_indices = [j for j in range(n_clients) if combination_bin[n_clients - 1 - j] == '1']
                clients_list = [j + 1 for j in coalition_indices]

                detail_record = {
                    'Trial': trial + 1,
                    'Combination': combination_bin,
                    'Clients': ",".join(str(c) for c in clients_list),
                    'Global Accuracy': nash_row['Global Accuracy'],
                    'Max Depth': max_depth
                }

                for client in range(1, n_clients + 1):
                    if client in clients_list:
                        detail_record[f'Client {client} Accuracy'] = client_global_accuracies[client - 1]
                        val = shapley_df.loc[shapley_df['Client'] == f'Client {client}', 'Normalized Shapley Value']
                        detail_record[f'Client {client} Shapley Value'] = val.values[0] if not val.empty else np.nan
                    else:
                        detail_record[f'Client {client} Accuracy'] = np.nan
                        detail_record[f'Client {client} Shapley Value'] = np.nan
                nash_details_all.append(detail_record)

            results.extend(trial_results)
        
        df_results = pd.DataFrame(results)
        df_results['Max Depth'] = max_depth
        all_results.append(df_results)
        
        df_nash_counts = pd.DataFrame(nash_equilibrium_counts.items(), columns=['Nash Equilibrium', 'Occurrences'])
        df_nash_counts['Max Depth'] = max_depth
        all_nash_equilibrium_counts.append(df_nash_counts)
        
        filename_counts = os.path.join(save_dir, f"Nash_Equilibrium_Counts_SpambaseFedFor_max_depth_{max_depth}.csv")
        df_nash_counts.to_csv(filename_counts, index=False)
        
        df_nash_details = pd.DataFrame(nash_details_all)
        all_nash_details.append(df_nash_details)

    final_nash_counts_df = pd.concat(all_nash_equilibrium_counts, ignore_index=True)
    final_nash_details_df = pd.concat(all_nash_details, ignore_index=True)

    final_nash_counts_df.to_csv(os.path.join(save_dir, "Nash_Equilibrium_Counts.csv"), index=False)
    final_nash_details_df.to_csv(os.path.join(save_dir, "Nash_Equilibrium_Details_SpambaseFedFor.csv"), index=False)

    print("Process complete. Results saved.")


In [ ]:
save_dir = '.../results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedFor/3_Corrupted_Clients'
results = run_experiment_2(n_trials=10,  n_clients=10, max_depths=[10, 100], partitions_corrupted= 3, corrupt_client_indices= [0,1,2], df=df , save_dir=save_dir, base_random_seed=42)


Running experiment with max_depth = 10
  Trial 1/10 for max_depth = 10
Corrupted client indices: [1 2 3]


    Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:00<00:00, 1046.09it/s]


  Trial 2/10 for max_depth = 10
Corrupted client indices: [1 2 3]


    Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:00<00:00, 1032.30it/s]


  Trial 3/10 for max_depth = 10
Corrupted client indices: [1 2 3]


...

  Trial 9/10 for max_depth = 100
Corrupted client indices: [1 2 3]


    Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [00:01<00:00, 1009.81it/s]


  Trial 10/10 for max_depth = 100
Corrupted client indices: [1 2 3]


    Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [00:01<00:00, 1017.37it/s]


Process complete. Results saved.


In [ ]:
save_dir = '.../results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedFor/4_Corrupted_Clients'
results = run_experiment_2(n_trials=10,  n_clients=10, max_depths=[10, 100], partitions_corrupted= 4, corrupt_client_indices= [0,1,2,3], df=df , save_dir=save_dir, base_random_seed=42)


Running experiment with max_depth = 10
  Trial 1/10 for max_depth = 10
Corrupted client indices: [1 2 3 4]


    Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:00<00:00, 1039.10it/s]


  Trial 2/10 for max_depth = 10
Corrupted client indices: [1 2 3 4]


    Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:00<00:00, 1042.54it/s]


  Trial 3/10 for max_depth = 10
Corrupted client indices: [1 2 3 4]


...

  Trial 9/10 for max_depth = 100
Corrupted client indices: [1 2 3 4]


    Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [00:01<00:00, 1009.05it/s]


  Trial 10/10 for max_depth = 100
Corrupted client indices: [1 2 3 4]


    Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [00:01<00:00, 991.48it/s]


Process complete. Results saved.


In [ ]:
save_dir = '.../results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedFor/6_Corrupted_Clients'
results = run_experiment_2(n_trials=10,  n_clients=10, max_depths=[10, 100], partitions_corrupted= 6, corrupt_client_indices= [0,1,2,3,4,5], df=df , save_dir=save_dir, base_random_seed=42)


Running experiment with max_depth = 10
  Trial 1/10 for max_depth = 10
Corrupted client indices: [1 2 3 4 5 6]


    Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:01<00:00, 1016.80it/s]


  Trial 2/10 for max_depth = 10
Corrupted client indices: [1 2 3 4 5 6]


    Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:00<00:00, 1032.53it/s]


  Trial 3/10 for max_depth = 10
Corrupted client indices: [1 2 3 4 5 6]


...

  Trial 9/10 for max_depth = 100
Corrupted client indices: [1 2 3 4 5 6]


    Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [00:01<00:00, 999.90it/s]


  Trial 10/10 for max_depth = 100
Corrupted client indices: [1 2 3 4 5 6]


    Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [00:01<00:00, 1016.46it/s]


Process complete. Results saved.


In [ ]:
save_dir = '.../results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedFor/8_Corrupted_Clients'
results = run_experiment_2(n_trials=10,  n_clients=10, max_depths=[10, 100], partitions_corrupted= 8, corrupt_client_indices= [0,1,2,3,4,5,6,7], df=df , save_dir=save_dir, base_random_seed=42)


Running experiment with max_depth = 10
  Trial 1/10 for max_depth = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]


    Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:01<00:00, 973.77it/s]


  Trial 2/10 for max_depth = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]


    Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:01<00:00, 1013.25it/s]


  Trial 3/10 for max_depth = 10
Corrupted client indices: [1 2 3 4 5 6 7 8]


...

  Trial 9/10 for max_depth = 100
Corrupted client indices: [1 2 3 4 5 6 7 8]


    Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [00:01<00:00, 1015.93it/s]


  Trial 10/10 for max_depth = 100
Corrupted client indices: [1 2 3 4 5 6 7 8]


    Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [00:01<00:00, 1021.15it/s]


Process complete. Results saved.


In [ ]:
save_dir = '.../results/fedLR_FedFor__with_Corrupting_Spambase_Nash_Check/FedFor/9_Corrupted_Clients'
results = run_experiment_2(n_trials=10,  n_clients=10, max_depths=[10, 100], partitions_corrupted= 9, corrupt_client_indices= [0,1,2,3,4,5,6,7,8], df=df , save_dir=save_dir, base_random_seed=42)


Running experiment with max_depth = 10
  Trial 1/10 for max_depth = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]


    Evaluating coalitions for trial 1: 100%|██████████| 1023/1023 [00:00<00:00, 1033.31it/s]


  Trial 2/10 for max_depth = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]


    Evaluating coalitions for trial 2: 100%|██████████| 1023/1023 [00:01<00:00, 978.98it/s]


  Trial 3/10 for max_depth = 10
Corrupted client indices: [1 2 3 4 5 6 7 8 9]


...

  Trial 9/10 for max_depth = 100
Corrupted client indices: [1 2 3 4 5 6 7 8 9]


    Evaluating coalitions for trial 9: 100%|██████████| 1023/1023 [00:01<00:00, 1020.48it/s]


  Trial 10/10 for max_depth = 100
Corrupted client indices: [1 2 3 4 5 6 7 8 9]


    Evaluating coalitions for trial 10: 100%|██████████| 1023/1023 [00:01<00:00, 1015.94it/s]


Process complete. Results saved.
